# Problemas de prueba de hipótesis

## Ejercicio 1

Usted es un nutricionista que investiga dos tipos diferentes de dietas para ver si existe una diferencia significativa en la pérdida de peso después de un mes. Eliges dos grupos aleatorios de personas; un grupo sigue la primera dieta y el otro sigue la segunda. Al final del mes, se registra la pérdida de peso (en kg) de cada persona:

| Diet 1 | Diet 2 |
|:-------|:-------|
| 2.0 | 3.0 |
| 2.5 | 3.2 |
| 3.0 | 3.1 |
| 2.8 | 2.9 |
| 2.3 | 2.8 |
| 2.7 | 3.0 |
| 2.5 | 3.2 |

Con estos datos se busca responder a la siguiente pregunta: ¿Existe una diferencia significativa en la pérdida de peso promedio entre las personas que siguieron la primera dieta y las que siguieron la segunda dieta?

Para sacar conclusiones, siga los siguientes puntos:

-Enunciar la hipótesis: hipótesis nula y alternativa.
-Realizar la prueba para comprobar la hipótesis. Puede utilizar la prueba t de Student.
-Analizar las conclusiones.

In [77]:
import pandas as pd
dietas = {'Diet1': [2.0, 2.5, 3.0, 2.8, 2.3, 2.7, 2.5],
          'Diet2': [3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2]}
dietas = pd.DataFrame(dietas)
print (dietas)

   Diet1  Diet2
0    2.0    3.0
1    2.5    3.2
2    3.0    3.1
3    2.8    2.9
4    2.3    2.8
5    2.7    3.0
6    2.5    3.2


In [78]:
dieta1 = dietas[['Diet1']]
dieta2 = dietas[['Diet2']]

In [79]:
# Planteo hipótesis
H0_t = 'No hay una diferencia significativa en la perdida de peso para cada dieta'
H1_t = 'Hay una diferencia significativa en la perdida de peso para cada dieta'

In [80]:
# Dos muestras T-Test
from scipy.stats import ttest_ind

t_stat, pvalue = ttest_ind (dieta1, dieta2)
print (t_stat, pvalue)

[-3.5383408] [0.00408327]


In [81]:
# Conclusión

if pvalue < 0.05:
    print (f'Rechazo H0. {H1_t}')
else: 
    print (f'Acepto H0. {H0_t}')

Rechazo H0. Hay una diferencia significativa en la perdida de peso para cada dieta


## ANOVA

**ANOVA** (*Analysis of Variance - Análisis de variación*) Es una técnica estadística utilizada para comparar las medidas de dos o más grupos. La idea detrás de ANOVA es descomponer la variabilidad total de los datos en dos componentes: variabilidad entre grupos y variabilidad dentro de los grupos:

- **Variabilidad entre grupos**: Esta variabilidad se refiere a las diferencias entre las medias del grupo. Si esta variabilidad es considerablemente mayor que la variabilidad dentro del grupo, podría ser una indicación de que al menos una de las medias del grupo es diferente.
- **Variabilidad dentro del grupo**: Esta variabilidad se refiere a la dispersión de los datos dentro de cada grupo. Si todos los grupos tienen una variabilidad similar, entonces cualquier diferencia notable en las medias de los grupos podría considerarse significativa.

Las hipótesis en ANOVA suelen incluir:

- **Hipótesis nula** ($H₀$): Las medias de todos los grupos son iguales.
- **Hipótesis alternativa** ($H₁$): Al menos una de las medias del grupo es diferente.

Si el resultado de la prueba ANOVA es significativo (por ejemplo, un valor p inferior a un umbral como 0,05), esto sugiere que al menos la media de un grupo es diferente.

## Ejercicio 2

Un agricultor decide probar tres tipos diferentes de fertilizantes para determinar si alguno es superior en términos de producción de maíz. El agricultor planta maíz en 15 parcelas idénticas y utiliza los tres fertilizantes (5 parcelas para cada tipo). Al final de la temporada mide el rendimiento de maíz (en kg) de cada parcela, con el siguiente resultado:

| Fertilizante 1 | Fertilizante 2 | Fertilizante 3 |
|:-------------|:-------------|:-------------|
| 20 | 22 | 24 |
| 21 | 21 | 23 |
| 20 | 23 | 22 |
| 19 | 22 | 23 |
| 20 | 21 | 24 |

Con estos datos busca responder la siguiente pregunta: ¿Existe una diferencia significativa en el rendimiento promedio del maíz entre los tres tipos de fertilizantes?

Para ayudarte, sigue los siguientes puntos:

-Enunciar la hipótesis: hipótesis nula y alternativa.
-Realizar la prueba ANOVA.
-Analizar las conclusiones.
-Si un fertilizante es mejor que otro, ¿cómo podemos saberlo?

In [82]:
# Paso datos de fertilizantes
f1 = [20, 21, 20, 19, 20]
f2 = [22, 21, 23, 22, 21]
f3 = [24, 23, 22, 23, 24]

In [83]:
# Planteo hipótesis

H0_anova = 'No hay una diferencia significativa en el rendimiento de maiz de los fertilizantes'
H1_anova = 'Hay una diferencia significativa en el rendimiento de maiz de los fertilizantes'

In [84]:
import scipy.stats as stats
f_stat, pvalue2 = stats.f_oneway(f1, f2, f3)

print(f_stat, pvalue2)


20.315789473684188 0.00014047824793190472


In [85]:
# Conclusión

if pvalue2 < 0.05:
    print (f'Rechazo H0. {H1_anova}')
else: 
    print (f'Acepto H0. {H0_anova}')

Rechazo H0. Hay una diferencia significativa en el rendimiento de maiz de los fertilizantes


In [86]:
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd

data = np.concatenate([f1, f2, f3])
print (data)


[20 21 20 19 20 22 21 23 22 21 24 23 22 23 24]


In [87]:
labels = ["F1"] * 5 + ["F2"] * 5 + ["F3"] * 5

# Tukey test
result = pairwise_tukeyhsd(data, labels, alpha = 0.05)
print(result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    F1     F2      1.8 0.0099 0.4572 3.1428   True
    F1     F3      3.2 0.0001 1.8572 4.5428   True
    F2     F3      1.4 0.0409 0.0572 2.7428   True
--------------------------------------------------


El tercer fertilizante es el de mejores resultados siendo las diferencias de medias entre todos los fertilizantes significativas
